In [0]:
from pyspark.sql.functions import struct, collect_list, to_json,map_from_entries,collect_list
from pyspark.sql.types import *

In [0]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("custname", StringType(), True),
    StructField("item", StringType(), True),
    StructField("quantity", IntegerType(), True)
])

# Sample data with repeating customers
data = [
    (1, 'John', 'ProductA', 5),
    (2, 'Alice', 'ProductB', 3),
    (3, 'Bob', 'ProductC', 8),
    (1, 'John', 'ProductD', 2),
    (2, 'Alice', 'ProductE', 6),
    (1, 'John', 'ProductF', 4)
]

# Create DataFrame using the defined schema and data
df = spark.createDataFrame(data, schema=schema)
df.display()

id,custname,item,quantity
1,John,ProductA,5
2,Alice,ProductB,3
3,Bob,ProductC,8
1,John,ProductD,2
2,Alice,ProductE,6
1,John,ProductF,4


In [0]:
df2 = df.groupBy("id", "custname").agg(map_from_entries(collect_list(struct("item", "quantity"))).alias("Purchases"))
df2.show(truncate=False)



+---+--------+---------------------------------------------+
|id |custname|Purchases                                    |
+---+--------+---------------------------------------------+
|1  |John    |{ProductA -> 5, ProductD -> 2, ProductF -> 4}|
|2  |Alice   |{ProductB -> 3, ProductE -> 6}               |
|3  |Bob     |{ProductC -> 8}                              |
+---+--------+---------------------------------------------+



In [0]:
my_format = [row.asDict(recursive=True) for row in df2.collect() ]
print(my_format)

[{'id': 1, 'custname': 'John', 'Purchases': {'ProductA': 5, 'ProductF': 4, 'ProductD': 2}}, {'id': 2, 'custname': 'Alice', 'Purchases': {'ProductE': 6, 'ProductB': 3}}, {'id': 3, 'custname': 'Bob', 'Purchases': {'ProductC': 8}}]


In [0]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("custname", StringType(), True),
    StructField("item", StringType(), True)
])

# Sample data with repeating customers
data = [
    (1, 'John', 'ProductA'),
    (2, 'Alice', 'ProductB'),
    (3, 'Bob', 'ProductC'),
    (1, 'John', 'ProductD'),
    (2, 'Alice', 'ProductE'),
    (1, 'John', 'ProductF')
]

# Create DataFrame using the defined schema and data
df = spark.createDataFrame(data, schema=schema)
df.display()
df2 = df.groupBy("id", "custname").agg(map_from_entries(collect_list(struct("item"))).alias("Purchases"))
df2.show(truncate=False)

id,custname,item
1,John,ProductA
2,Alice,ProductB
3,Bob,ProductC
1,John,ProductD
2,Alice,ProductE
1,John,ProductF


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4382232317541909>:20
     18 df = spark.createDataFrame(data, schema=schema)
     19 df.display()
---> 20 df2 = df.groupBy("id", "custname").agg(map_from_entries(collect_list(struct("item"))).alias("Purchases"))
     21 df2.show(truncate=False)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/group.py:177, in GroupedData.agg(self, *exprs)
    175     assert all(isinstance(c, Column) for c in exprs), "all exprs should be Column"
    176     exprs = cast(Tuple[Colu

In [0]:
from pyspark.sql.functions import struct, collect_list, expr
data = [
    (1, 'John', 'ProductA', 5),
    (2, 'Alice', 'ProductB', 3),
    (3, 'Bob', 'ProductC', 8),
    (1, 'John', 'ProductD', 2),
    (2, 'Alice', 'ProductE', 6),
    (1, 'John', 'ProductF', 4)
]
schema = ["id", "custname", "item", "quantity"]
df = spark.createDataFrame(data, schema=schema)

In [0]:
# Group by 'custname' and aggregate purchases as a map
df2 = df.groupBy("id", "custname").agg(
    expr("map_from_entries(collect_list(struct(item, quantity)))").alias("Purchases"))
# Show the aggregated DataFrame
df2.show(truncate=False)

+---+--------+---------------------------------------------+
|id |custname|Purchases                                    |
+---+--------+---------------------------------------------+
|1  |John    |{ProductA -> 5, ProductD -> 2, ProductF -> 4}|
|2  |Alice   |{ProductB -> 3, ProductE -> 6}               |
|3  |Bob     |{ProductC -> 8}                              |
+---+--------+---------------------------------------------+

